# Example notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import anatomize as an
import spatialdata as sd
from zarr.errors import PathNotFoundError
import hvplot.xarray  # noqa

With myst it is possible to link in the text cell of a notebook such as this one the documentation of a function or a class.

Let's take as an example the function {func}`anatomize.exposure.adjust_gamma`. 
You can see that by clicking on the text, the link redirects to the API documentation of the function. 
Check the raw markdown of this cell to understand how this is specified.

This works also for any package listed by `intersphinx`. Go to `docs/conf.py` and look for the `intersphinx_mapping` variable. 
There, you will see a list of packages (that this package is dependent on) for which this functionality is supported. 

For instance, we can link to the class {class}`anndata.AnnData`, to the attribute {attr}`anndata.AnnData.obs` or the method {meth}`anndata.AnnData.write`.

Again, check the raw markdown of this cell to see how each of these links are specified.

You can read more about this in the [intersphinx page](https://www.sphinx-doc.org/en/master/usage/extensions/intersphinx.html) and the [myst page](https://myst-parser.readthedocs.io/en/v0.15.1/syntax/syntax.html#roles-an-in-line-extension-point).

In [3]:
try:
    blobs_sdata = sd.read_zarr("blobs.zarr")
except PathNotFoundError:
    blobs_sdata = sd.datasets.blobs(length=2048, n_channels=100)
    blobs_sdata.write("blobs.zarr")
blobs_sdata = sd.read_zarr("blobs.zarr")

In [4]:
blobs_image = blobs_sdata.images["blobs_image"]

In [5]:
blobs_image = blobs_image.chunk(chunks={"c": 1, "y": 2048, "x": 2048})

In [6]:
an.exposure.adjust_gamma(blobs_image, gamma=1.5, gain=2.0).compute()

<xarray.DataArray (c: 100, y: 2048, x: 2048)> Size: 3GB
array([[[1.90954517e+00, 1.90953298e+00, 1.90950792e+00, ...,
         1.59973285e-03, 1.58461650e-03, 1.57708217e-03],
        [1.90925898e+00, 1.90924669e+00, 1.90922143e+00, ...,
         1.59920941e-03, 1.58409800e-03, 1.57656614e-03],
        [1.90868663e+00, 1.90867413e+00, 1.90864848e+00, ...,
         1.59816292e-03, 1.58306140e-03, 1.57553446e-03],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.88280189e+00, 1.88240088e+00, 1.88220008e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.88363840e+00, 1.88323680e+00, 1.88303571e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.88405694e+00, 1.88365506e+00, 1.88345382e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.33139077e+00, 1.33137370e+00, 1.33136487e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.33069022e+00, 1.33067317e+00, 1.33066434e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.32929024e+00, 1.32927320e+00, 1.32926439e+00],
...
        [2.43790699e-09, 2.45836451e-09, 2.49953263e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.40720226e-09, 2.42740212e-09, 2.46805175e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.39191892e-09, 2.41199053e-09, 2.45238207e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[3.34712979e-03, 3.34521528e-03, 3.34138954e-03, ...,
         5.21597630e-01, 5.22146946e-01, 5.22421820e-01],
        [3.36266508e-03, 3.36074168e-03, 3.35689818e-03, ...,
         5.21982906e-01, 5.22532628e-01, 5.22807705e-01],
        [3.39383044e-03, 3.39188922e-03, 3.38801010e-03, ...,
         5.22753116e-01, 5.23303648e-01, 5.23579131e-01],
        ...,
        [2.04528260e-04, 2.05771917e-04, 2.08269820e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.04477487e-04, 2.05720835e-04, 2.08218118e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.04452039e-04, 2.05695232e-04, 2.08192205e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]])
Coordinates:
  * c        (c) int64 800B 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
  * y        (y) float64 16kB 0.5 1.5 2.5 3.5 ... 2.046e+03 2.046e+03 2.048e+03
  * x        (x) float64 16kB 0.5 1.5 2.5 3.5 ... 2.046e+03 2.046e+03 2.048e+03

In [7]:
import skimage.exposure as se

In [8]:
se.adjust_gamma(blobs_image, gamma=1.5, gain=2.0).compute()

<xarray.DataArray 'image' (c: 100, y: 2048, x: 2048)> Size: 3GB
array([[[1.90954517e+00, 1.90953298e+00, 1.90950792e+00, ...,
         1.59973285e-03, 1.58461650e-03, 1.57708217e-03],
        [1.90925898e+00, 1.90924669e+00, 1.90922143e+00, ...,
         1.59920941e-03, 1.58409800e-03, 1.57656614e-03],
        [1.90868663e+00, 1.90867413e+00, 1.90864848e+00, ...,
         1.59816292e-03, 1.58306140e-03, 1.57553446e-03],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.88280189e+00, 1.88240088e+00, 1.88220008e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.88363840e+00, 1.88323680e+00, 1.88303571e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.88405694e+00, 1.88365506e+00, 1.88345382e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.33139077e+00, 1.33137370e+00, 1.33136487e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.33069022e+00, 1.33067317e+00, 1.33066434e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.32929024e+00, 1.32927320e+00, 1.32926439e+00],
...
        [2.43790699e-09, 2.45836451e-09, 2.49953263e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.40720226e-09, 2.42740212e-09, 2.46805175e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.39191892e-09, 2.41199053e-09, 2.45238207e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[3.34712979e-03, 3.34521528e-03, 3.34138954e-03, ...,
         5.21597630e-01, 5.22146946e-01, 5.22421820e-01],
        [3.36266508e-03, 3.36074168e-03, 3.35689818e-03, ...,
         5.21982906e-01, 5.22532628e-01, 5.22807705e-01],
        [3.39383044e-03, 3.39188922e-03, 3.38801010e-03, ...,
         5.22753116e-01, 5.23303648e-01, 5.23579131e-01],
        ...,
        [2.04528260e-04, 2.05771917e-04, 2.08269820e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.04477487e-04, 2.05720835e-04, 2.08218118e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.04452039e-04, 2.05695232e-04, 2.08192205e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]])
Coordinates:
  * c        (c) int64 800B 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
  * y        (y) float64 16kB 0.5 1.5 2.5 3.5 ... 2.046e+03 2.046e+03 2.048e+03
  * x        (x) float64 16kB 0.5 1.5 2.5 3.5 ... 2.046e+03 2.046e+03 2.048e+03

In [14]:
an.exposure.adjust_gamma(blobs_image, gamma=1.5, gain=2.0).pipe(an.exposure.adjust_log, gain=2.0).compute()

<xarray.DataArray (c: 100, y: 2048, x: 2048)> Size: 3GB
array([[[2.13599354e+00, 2.13598516e+00, 2.13596794e+00, ...,
         3.19690928e-03, 3.16672463e-03, 3.15167975e-03],
        [2.13579681e+00, 2.13578835e+00, 2.13577099e+00, ...,
         3.19586407e-03, 3.16568928e-03, 3.15064932e-03],
        [2.13540330e+00, 2.13539471e+00, 2.13537707e+00, ...,
         3.19377443e-03, 3.16361935e-03, 3.14858922e-03],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         2.11752540e+00, 2.11724717e+00, 2.11710784e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         2.11810566e+00, 2.11782711e+00, 2.11768761e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         2.11839593e+00, 2.11811722e+00, 2.11797764e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.69292997e+00, 1.69291533e+00, 1.69290776e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.69232892e+00, 1.69231428e+00, 1.69230671e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.69112721e+00, 1.69111258e+00, 1.69110501e+00],
...
        [4.87581397e-09, 4.91672880e-09, 4.99906516e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.81440442e-09, 4.85480411e-09, 4.93610352e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.78383776e-09, 4.82398121e-09, 4.90476414e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[6.68308124e-03, 6.67926499e-03, 6.67163900e-03, ...,
         8.39521711e-01, 8.40243606e-01, 8.40604740e-01],
        [6.71404793e-03, 6.71021402e-03, 6.70255276e-03, ...,
         8.40028056e-01, 8.40750301e-01, 8.41111610e-01],
        [6.77616880e-03, 6.77229947e-03, 6.76456745e-03, ...,
         8.41039914e-01, 8.41762859e-01, 8.42124517e-01],
        ...,
        [4.09014693e-04, 4.11501497e-04, 4.16496270e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.08913168e-04, 4.11399354e-04, 4.16392888e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.08862283e-04, 4.11348160e-04, 4.16341072e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]])
Coordinates:
  * c        (c) int64 800B 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
  * y        (y) float64 16kB 0.5 1.5 2.5 3.5 ... 2.046e+03 2.046e+03 2.048e+03
  * x        (x) float64 16kB 0.5 1.5 2.5 3.5 ... 2.046e+03 2.046e+03 2.048e+03